# TP 2 - Inferencia y Estimacion

Antonio Santiago Tepsich - Maximo Gubitosi

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv

# Extracción de Datos

In [2]:
def leer_archivo_csv(archivo_csv):
    # Lista para almacenar los datos del archivo
    altosGp1 = []
    anchosGp1 = []
    altosGp2 = []
    anchosGp2 = []
    gp1=[]
    gp2=[]

    try:
        with open(archivo_csv, 'r', newline='') as csv_file:
            csv_reader = csv.reader(csv_file)
            next(csv_reader)

            # Leer los datos del archivo
            for row in csv_reader:
                # Si la primera columna tiene valores numéricos, la segunda cadenas y la tercera flotantes
                if(row[0] == '' or row[1] == '' or row[2] == ''):
                    continue
                if(row[0]=="1"):
                    largo = int(row[1])
                    alto = int(row[2])
                    altosGp1.append(largo)
                    anchosGp1.append(alto)
                else:
                    largo = int(row[1])
                    alto = int(row[2])
                    altosGp2.append(largo)
                    anchosGp2.append(alto)
            
            gp1.append(altosGp1)
            gp1.append(anchosGp1)
            gp2.append(altosGp2)
            gp2.append(anchosGp2)
    
    except FileNotFoundError:
        print(f"El archivo '{archivo_csv}' no fue encontrado.")
        return None

    return gp1,gp2

## Entropía

 En el contexto de la teoría de la información, la entropía mide la cantidad de información que se espera obtener de una nueva muestra de datos.

 $$ H(p) = -\sum_{i} p_i \log_2(p_i) $$

In [3]:
def entropia(p):
    for i in range(len(p)):
        if(p[i]==0):
            p[i]=1
    return -np.sum(p*np.log2(p))

### Grupo 1

In [6]:
gp1,gp2 = leer_archivo_csv('Todo.csv')

matriz1 = np.histogram2d(gp1[0],gp1[1],bins=5)

matriz1 = matriz1[0]/np.sum(matriz1[0])

marginal_largo_gp1 = np.sum(matriz1,axis=1)
marginal_ancho_gp1 = np.sum(matriz1,axis=0)

# --------Calculo probabilidad del largo de gp1---------------- 
print("Entropia largo gp1: ",entropia(marginal_largo_gp1))

# --------Calculo probabilidad del ancho de gp1----------------
print("Entropia alto gp1: ",entropia(marginal_ancho_gp1))


Entropia largo gp1:  1.7206031643621587
Entropia alto gp1:  1.8415549118707901


### Grupo 2

In [7]:
gp1,gp2 = leer_archivo_csv('Todo.csv')

matriz2 = np.histogram2d(gp2[0],gp2[1],bins=5)
matriz2 = matriz2[0]/np.sum(matriz2[0])


marginal_largo_gp2 = np.sum(matriz2,axis=1)
marginal_ancho_gp2 = np.sum(matriz2,axis=0)

# --------Calculo probabilidad del largo de gp2----------------
print("Entropia largo gp2: ",entropia(marginal_largo_gp2))

# --------Calculo probabilidad del ancho de gp2----------------
print("Entropia alto gp2: ",entropia(marginal_ancho_gp2))

Entropia largo gp2:  1.728481135079078
Entropia alto gp2:  1.7906927146926772


### Conclusión

Estos valores representan la cantidad de información (o incertidumbre) en las mediciones de las hojas. Al comparar las entropías de gp1 y gp2, podemos observar que las mediciones de gp1 presentan una mayor incertidumbre (o desorden) en comparación con las de gp2.

## Entropía Condicional

La entropía condicional mide la incertidumbre promedio de una variable aleatoria X dado que el valor de otra variable aleatoria Y es conocido. Se utiliza para entender la cantidad de información que queda por conocer acerca de X después de observar Y.

$$ H(X|Y) = \sum_{y \in Y} p(y) \sum_{x \in X} p(x|y) \log_2 \left(\frac{1}{p(x|y)}\right) $$


In [8]:
def conditonal_entropy(x, y, bins):
    if len(x)<len(y):
        #elegir randoms de y
        y = np.random.choice(y, len(x))
    elif len(y)<len(x):
        #elegir randoms de x
        x = np.random.choice(x, len(y))

    
    hist_2d, x_edges, y_edges = np.histogram2d(x, y, bins=bins)    
    hist_2d = hist_2d/np.sum(hist_2d) #Matriz de probabilidades

    conditonal_entropy = 0
    for i in range (len(x_edges)-1):
        for j in range (len(y_edges)-1):
            if hist_2d[i,j] != 0:
                conditonal_entropy -= hist_2d[i,j]*np.log2(hist_2d[i,j]/hist_2d[i].sum())

    return conditonal_entropy


In [9]:
gp1,gp2 = leer_archivo_csv('Todo.csv')

# --------Calculo la entropia condicional de A1 dado L1----------------
print("H(A1/L1): ",conditonal_entropy(gp1[0], gp1[1], 5))

# --------Calculo la entropia condicional de A2 dado L2----------------
print("H(A2/L2): ",conditonal_entropy(gp2[0], gp2[1], 5))

# --------Calculo la entropia condicional de L1 dado A1----------------
print("H(A1/A2): ",conditonal_entropy(gp2[1], gp1[1], 5))

# --------Calculo la entropia condicional de L2 dado A2----------------
print("H(L1/L2): ",conditonal_entropy(gp2[0], gp1[0], 5))

H(A1/L1):  1.3115433811024402
H(A2/L2):  1.1251274508998534
H(A1/A2):  1.744401306971017
H(L1/L2):  1.5389123759431875


### Conclusión

Estas medidas nos proporcionan información sobre cómo el conocimiento de una de las variables (como el largo de una hoja) puede influir en la incertidumbre de otra variable (como el alto de la misma hoja o de otra hoja).

## Información Mutua

La información mutua entre dos variables aleatorias X e Y mide la cantidad de información que se puede obtener sobre una variable al observar la otra.

$$
I(X;Y) = H(X) - H(X|Y) = \sum_{x \in X, y \in Y} p(x,y) \log_2 \left(\frac{p(x,y)}{p(x)p(y)}\right)
$$


In [10]:
gp1,gp2 = leer_archivo_csv('Todo.csv')

# I(A/L) = H(A) - H(A/L)


matriz1 = np.histogram2d(gp1[0],gp1[1],bins=5)

matriz1 = matriz1[0]/np.sum(matriz1[0])


#--------Calculon probabilidad del ancho de gp1---------------- 
marginal_largo_gp1 = np.sum(matriz1,axis=1)

#--------Calculon probabilidad del alto de gp2----------------
marginal_ancho_gp1 = np.sum(matriz1,axis=0)

# ----------Informacion Mutua entre A1 y L1----------------
print("I(A1/L1): ",entropia(marginal_largo_gp1)-conditonal_entropy(gp1[0], gp1[1], 5))

# ----------Informacion Mutua entre A2 y L2----------------
print("I(A2/L2): ",entropia(marginal_ancho_gp1)-conditonal_entropy(gp2[0], gp2[1], 5))

I(A1/L1):  0.4090597832597185
I(A2/L2):  0.7164274609709367


### Conclusión

Para ambos grupos hay una cantidad significativa de información compartida entre el largo y ancho de las hojas en cada grupo especifico. Algo que nos resulta interesante, ya que si conocemos el valor de uno de estos atributos, podemos hacer una predicción razonablemente precisa sobre el otro atributo.


## Distancia Kullback–Leibler

La divergencia de Kullback-Leibler es una medida de cuán diferente es una distribución probabilística p de otra distribución q.

$$D(p(x,y) || q(x,y)) = \sum_{x} \sum_{y} p(x,y) \log \left( \frac{p(x,y)}{q(x,y)} \right)
$$

In [11]:
# ----------Distancia de Kullback-Leibler entre gp1 y gp2----------------
def kullback_leibler(p, q):
    if len(p[0]) > len(q[0]):
        #elegir randoms de q
        q[0] = np.random.choice(q[0], len(p[0]))
    elif len(q[0]) > len(p[0]):
        #elegir randoms de p
        p[0] = np.random.choice(p[0], len(q[0]))

    if len(p[1]) > len(q[1]):
        #elegir randoms de q
        q[1] = np.random.choice(q[1], len(p[1]))
    elif len(q[1]) > len(p[1]):
        #elegir randoms de p
        p[1] = np.random.choice(p[1], len(q[1]))

    p_hist_2d, x_edges, y_edges = np.histogram2d(p[0], p[1], bins=5)
    p_hist_2d = p_hist_2d/np.sum(p_hist_2d) #Matriz de probabilidades

    q_hist_2d, x_edges, y_edges = np.histogram2d(q[0], q[1], bins=5)
    q_hist_2d = q_hist_2d/np.sum(q_hist_2d) #Matriz de probabilidades

    suma=0
    for i in range(len(x_edges)-1):
        for j in range(len(y_edges)-1):
            p1=p_hist_2d[i][j]
            q1=q_hist_2d[i][j]
            if q1!=0 and p1!=0:
                suma+=p1*np.log2(p1/q1)
    return suma

In [60]:
gp1,gp2 = leer_archivo_csv('Todo.csv')

print("D(gp1||gp2): ", kullback_leibler(gp1, gp2))

D(gp1||gp2):  0.9325444291891447


### Conclusión

EL resultado obtenido sugiere que existe una diferencia significativa en la distribución de las medidas de ancho y largo entre los dos grupos de hojas. Específicamente, este valor indica cuánto difieren las probabilidades del grupo gp1 respecto a las del grupo gp2 en términos de ancho y largo. Al ser un valor mayor a cero, esto sugiere que las distribuciones no son idénticas, y que gp1 se desvía de gp2 en una medida de 0.9131 en términos de su contenido de información.